In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os
from vzam.searcher import FaissRhashVideoSearcher
from sklearn.metrics import classification_report
%matplotlib inline

In [ ]:
initial_cwd = os.getcwd()
initial_cwd

In [4]:
print(initial_cwd)
os.chdir(initial_cwd)
os.chdir('..')
print(os.getcwd())

D:\projects\kr\vzam\notebooks
D:\projects\kr\vzam


In [6]:
from vzam import *

In [7]:
vcdb = pd.read_csv('data/vcdb.csv')

In [8]:
vcdb.head()

,query,source,source_id
0,core_dataset/baggio_penalty_1994\3504e360accba...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
1,core_dataset/baggio_penalty_1994\37b31d607d31a...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
2,core_dataset/baggio_penalty_1994\458db5aa227ae...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
3,core_dataset/baggio_penalty_1994\5c5714c0a56fd...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
4,core_dataset/baggio_penalty_1994\67824b87c0698...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [9]:
vcdb['query_id'] = vcdb['query'].apply(lambda x: os.path.basename(x).split('\\')[-1])
vcdb.head()

,query,source,source_id,query_id
0,core_dataset/baggio_penalty_1994\3504e360accba...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv,3504e360accbaccb1580befbb441f1019664c2bb.mp4
1,core_dataset/baggio_penalty_1994\37b31d607d31a...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv,37b31d607d31a47d347b15dae2b8aa63e57861eb.flv
2,core_dataset/baggio_penalty_1994\458db5aa227ae...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv,458db5aa227ae49ceb8bc1bed5f9cf5b4bed63f2.flv
3,core_dataset/baggio_penalty_1994\5c5714c0a56fd...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv,5c5714c0a56fd2a96f99db2f59b0d03659d77cdf.flv
4,core_dataset/baggio_penalty_1994\67824b87c0698...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv,67824b87c0698c499acad123b7498ae17f97bf6d.flv


In [10]:
vcdb_lut = vcdb[['query_id', 'source_id']]
vcdb_lut.index = vcdb_lut.query_id

In [11]:

vcdb_lut.ix['3504e360accbaccb1580befbb441f1019664c2bb.mp4'].source_id

'bb604f57a18455867544e79c2e32bf5583c358d4.flv'

In [12]:
SOURCE_DATASET = 'data/vcdb_rhash_source.csv'
QUERY_DATASET = 'data/vcdb_rhash_query.csv'

In [13]:
def list_converter(text):
    return eval(text.replace(' ', ','))

source_df = pd.read_csv(SOURCE_DATASET, converters={"feature": list_converter})
source_df.head()

,feature,ts,id
0,"[0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, ...",1.85,_bb604f57a18455867544e79c2e32bf5583c358d4.flv
1,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",3.85,_bb604f57a18455867544e79c2e32bf5583c358d4.flv
2,"[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ...",5.85,_bb604f57a18455867544e79c2e32bf5583c358d4.flv
3,"[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, ...",7.85,_bb604f57a18455867544e79c2e32bf5583c358d4.flv
4,"[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, ...",9.85,_bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [14]:
frame_times = source_df['ts']
labels = source_df.id.values
frame_vectors = np.vstack(source_df['feature'].values)
frame_vectors

array([[0, 0, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [121]:
searcher = FaissRhashVideoSearcher(frame_vectors, labels, frame_times)


In [122]:
query_df = pd.read_csv(QUERY_DATASET, converters={"feature": list_converter})
query_df.head()

,feature,ts,id,source_id
0,"[0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, ...",1.800,_3504e360accbaccb1580befbb441f1019664c2bb.mp4,bb604f57a18455867544e79c2e32bf5583c358d4.flv
1,"[0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...",3.800,_3504e360accbaccb1580befbb441f1019664c2bb.mp4,bb604f57a18455867544e79c2e32bf5583c358d4.flv
2,"[1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, ...",5.800,_3504e360accbaccb1580befbb441f1019664c2bb.mp4,bb604f57a18455867544e79c2e32bf5583c358d4.flv
3,"[1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, ...",7.800,_3504e360accbaccb1580befbb441f1019664c2bb.mp4,bb604f57a18455867544e79c2e32bf5583c358d4.flv
4,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",1.825,_37b31d607d31a47d347b15dae2b8aa63e57861eb.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [123]:
query_ids = query_df.id.unique()
len(query_ids)

519

In [126]:
searcher.dist_threshold = 15

In [127]:
true_labels = []
predictions = []
for i, query_id in enumerate(query_ids):
    tdf = query_df[query_df.id == query_id]
    true_label = vcdb_lut.ix[query_id.replace('_', '')].source_id
    true_labels.append('miss' if true_label is np.NaN else true_label)
    query_vectors = np.vstack(tdf['feature'].values)
    prediction = searcher.lookup_fun(query_vectors)
    predicted_label = prediction[0][0].replace('_', '') if prediction else 'miss'
    predictions.append(predicted_label)
    
    if i % 50 == 0:
        print(i)
print(classification_report(true_labels, predictions))

0
50
100
150
200
250
300
350
400
450
500
                                              precision    recall  f1-score   support

098df9c477ecf6c4f3818b198490b0258163015d.flv       0.50      0.18      0.27        11
127dab55025984673f65d3a23b1fea99ecc79b15.mp4       0.10      0.08      0.09        26
3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv       0.67      0.18      0.29        11
3df02ec5ddb74647d8fb96f08adba8e152050ef9.flv       0.44      1.00      0.61         7
410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv       0.80      0.67      0.73         6
46f2e964ae16f5c27fad70d6849c76616fad7502.flv       0.00      0.00      0.00         4
97c71f57e83f6ab16906fdd7edc8e3f88d2ceee1.flv       0.41      0.54      0.46        24
bb604f57a18455867544e79c2e32bf5583c358d4.flv       0.00      0.00      0.00        10
d2015b438b70f022967713d6f977ebc67a16839e.flv       0.00      0.00      0.00         7
                                        miss       0.84      0.84      0.84       413

           

In [117]:
import scipy
def lookup_sequence(clf, vectors):
    vectors = np.asarray(vectors).astype('uint8')
    D, I = clf.index.search(vectors, 10)
    labels = clf.labels[I]
    timestamps = clf.timestamps.values[I]
    # print(D)
    # print(I)
    sequence_starts = I[0]
    candidates = []
    for i, start_index in enumerate(sequence_starts):
        dist = D[0][i]
        if dist > clf.dist_threshold:
            continue
        candidate = {'start_index': start_index,
                     'dist': dist,
                     'label': clf.labels[start_index],
                     'ts': clf.timestamps.values[start_index],
                     }
        candidates.append(candidate)
    # print(candidates)
    # sequences = []
    # data_df = pd.DataFrame({'vector': vectors, 'label': labels}, index=range(len(vectors)))
    # print(data_df.head())
    seq_candidates = []
    for candidate in candidates:
        # Extract sequence for that candidate
        index_seq = slice(candidate['start_index']+1,candidate['start_index']+len(query_vectors)-1)
        label_sequence = clf.labels[index_seq]
        if len(np.unique(label_sequence)) != 1:
            # print('impossible candidate', candidate)
            # print(label_sequence)
            continue
        
        source_vectors = clf.vectors[index_seq]
        distances = []
        for qv, v in zip(vectors, source_vectors):
            distances.append(scipy.spatial.distance.hamming(qv, v))
        total_dist = (candidate['dist']/len(vectors[0]) + sum(distances))/len(vectors)
        seq_candidates.append({
            'start_index': candidate['start_index'],
            'ts': candidate['ts'],
            'label': candidate['label'],
            'dist': total_dist,
        })
    seq_candidates = sorted(seq_candidates, key=lambda x: x['dist'])
    return seq_candidates
        
lookup_sequence(searcher, query_vectors)

[{'start_index': 279,
  'ts': 137.825,
  'label': '_46f2e964ae16f5c27fad70d6849c76616fad7502.flv',
  'dist': 0.20833333333333334},
 {'start_index': 634,
  'ts': 3.825,
  'label': '_d2015b438b70f022967713d6f977ebc67a16839e.flv',
  'dist': 0.21875},
 {'start_index': 353,
  'ts': 1.85,
  'label': '_410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv',
  'dist': 0.22916666666666666},
 {'start_index': 729,
  'ts': 193.825,
  'label': '_d2015b438b70f022967713d6f977ebc67a16839e.flv',
  'dist': 0.23958333333333334},
 {'start_index': 531,
  'ts': 87.85,
  'label': '_098df9c477ecf6c4f3818b198490b0258163015d.flv',
  'dist': 0.25},
 {'start_index': 218,
  'ts': 15.825,
  'label': '_46f2e964ae16f5c27fad70d6849c76616fad7502.flv',
  'dist': 0.2604166666666667},
 {'start_index': 754,
  'ts': 243.825,
  'label': '_d2015b438b70f022967713d6f977ebc67a16839e.flv',
  'dist': 0.2604166666666667},
 {'start_index': 281,
  'ts': 141.825,
  'label': '_46f2e964ae16f5c27fad70d6849c76616fad7502.flv',
  'dist': 0.28125}]

In [118]:
true_labels = []
predictions = []
for i, query_id in enumerate(query_ids):
    tdf = query_df[query_df.id == query_id]
    true_label = vcdb_lut.ix[query_id.replace('_', '')].source_id
    true_labels.append('miss' if true_label is np.NaN else true_label)
    query_vectors = np.vstack(tdf['feature'].values)
    prediction = lookup_sequence(searcher, query_vectors)
    prediction = prediction[0] if prediction else None
    predicted_label = prediction['label'].replace('_', '') if prediction else 'miss'
    predictions.append(predicted_label)
    
    if i % 50 == 0:
        print(i)
print(classification_report(true_labels, predictions))

0
50
100
150
200
250
300
350
400
450
500
                                              precision    recall  f1-score   support

098df9c477ecf6c4f3818b198490b0258163015d.flv       0.05      0.09      0.06        11
127dab55025984673f65d3a23b1fea99ecc79b15.mp4       0.19      0.85      0.31        26
3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv       0.03      0.27      0.06        11
3df02ec5ddb74647d8fb96f08adba8e152050ef9.flv       0.00      0.00      0.00         7
410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv       0.08      0.33      0.13         6
46f2e964ae16f5c27fad70d6849c76616fad7502.flv       0.00      0.00      0.00         4
97c71f57e83f6ab16906fdd7edc8e3f88d2ceee1.flv       0.33      0.67      0.44        24
bb604f57a18455867544e79c2e32bf5583c358d4.flv       0.16      0.50      0.24        10
d2015b438b70f022967713d6f977ebc67a16839e.flv       0.03      0.29      0.05         7
                                        miss       0.89      0.23      0.36       413

           

In [119]:
# Majority vote
true_labels = []
predictions = []
for i, query_id in enumerate(query_ids):
    tdf = query_df[query_df.id == query_id]
    true_label = vcdb_lut.ix[query_id.replace('_', '')].source_id# tdf.source_id.unique()[0] or 'miss' 
    true_labels.append('miss' if true_label is np.NaN else true_label)
    query_vectors = np.vstack(tdf['feature'].values)
    prediction = searcher.lookup(query_vectors)
    predicted_label = prediction[0].replace('_', '') if prediction else 'miss'
    predictions.append(predicted_label)
    
    if i % 50 == 0:
        print(i)
print(classification_report(true_labels, predictions))

0
50
100
150
200
250
300
350
400
450
500
                                              precision    recall  f1-score   support

098df9c477ecf6c4f3818b198490b0258163015d.flv       0.27      0.64      0.38        11
127dab55025984673f65d3a23b1fea99ecc79b15.mp4       0.26      0.96      0.41        26
3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv       0.67      0.18      0.29        11
3df02ec5ddb74647d8fb96f08adba8e152050ef9.flv       0.33      1.00      0.50         7
410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv       0.40      0.67      0.50         6
46f2e964ae16f5c27fad70d6849c76616fad7502.flv       0.00      0.00      0.00         4
97c71f57e83f6ab16906fdd7edc8e3f88d2ceee1.flv       0.16      0.92      0.27        24
bb604f57a18455867544e79c2e32bf5583c358d4.flv       0.06      0.60      0.11        10
d2015b438b70f022967713d6f977ebc67a16839e.flv       0.03      0.43      0.05         7
                                        miss       0.00      0.00      0.00       413

           

(['3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv',
  '3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv',
  '3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv',
  '3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv',
  '3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv',
  '3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
  'miss',
